In [76]:
# Run this cell to import the necessary libraries and initiate an Elasticsearch object.
import csv
import os
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError
import datetime

import json
from IPython.core.display import display, HTML
import re

es = Elasticsearch()

def create_index(es, index, delete=True):
    INDEX_NAME=index
    if es.indices.exists(INDEX_NAME) and delete==True:
        print("deleting '%s' index..." % (INDEX_NAME))
        res = es.indices.delete(index = INDEX_NAME)
        print(" response: '%s'" % (res))
    # since we are running locally, use one shard and no replicas
    request_body = {
        "settings" : {
            "number_of_shards": 1,
            "number_of_replicas": 0
        }
    }
    print("creating '%s' index..." % (INDEX_NAME))
    res = es.indices.create(index = INDEX_NAME, body = request_body)
    print(" response: '%s'" % (res))

Onderzoek naar one to many relationships in dictionary. En query
Basisprobleem: kan maar 1 parent gebruiken en children kun je niet definieren

Probleem 1:

- 2 personen instanties van 1 persoon

probleem 2:

- 1 persoon hoort bij verschillende units


In [77]:
#define person instances. In the case of emigrants these are the persons on the cards

pi1 = {'id': 1,
        'naam': 'jan',
        'type':'persoon_instantie',
        '_parent': {'type':'unit',
                    'id':1}}

pi2 = {'id': 2,
        'naam': 'jan',
        'type':'persoon_instantie',
        '_parent': {'type':'unit',
                        'id':1}}

pi3 = {'id': 3,
        'naam': 'kees',
        'type':'persoon_instantie',
        '_parent': {'type':'unit',
                    'id':2}}

#define (identified) persons
persoon1 = {'id': 'I',
           'type':'persoon',
           'pi':[1, 2]}

persoon2 = {'id': 'II',
           'type':'persoon',
           'pi':[3]}

#define migrant units
unit1 = {'id': 1,
        'type': 'unit'}

unit2 = {'id': 2,
        'type': 'unit'}


#make collections of all types
pis = [pi1, pi2, pi3]
personen = [persoon1, persoon2]
units = [unit1, unit2]

#get instances per identified person
for ps in personen:
    dep_pers = [p for p in pis if p['id'] in ps['pi']]
    namen = [pi['naam'] for pi in dep_pers]
    unit = [u['id'] for u in units if u['id'] in [d['_parent']['id'] for d in dep_pers]]
    print "naam: {name}; unit: {unit}".format(name=namen, unit=unit) 

naam: ['jan', 'jan']; unit: [1]
naam: ['kees']; unit: [2]


In [78]:
# nu als elasticsearch mapping

# create index
index = 'test_migrant'

request_body = {
        "settings" : {
            "number_of_shards": 5,
            "number_of_replicas": 1
        },

       'mappings': {
            "travel": {},
            "unit": {"properties": {
                        'prs_achternaam': {"type":"string"},
                        'prs_initialen': {"type":"string"},
                        'prs_tussenvoegsel': {"type":"string"},
                        'prs_uuid': {"type":"string"}
                        }
                     },    
            "person_instance": {
                    "_parent": {"type": "unit"},
                    "properties":{}
                    },

            "person" : {}

          }
    }

INDEX_NAME=index
if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
print("creating 'example_index' index...")
es.indices.create(index = index, body = request_body)

# check mapping
es.indices.get_mapping(index = index)

def upload_bulk(index, typ, data_dict):
    bulk_data = []
    for item in enumerate(data_dict):
        bulk_data.append({'index':{'_type':typ, 
                                   '_index':index, 
                                   '_id':'%s' % item[0]}})
        bulk_data.append(item[1])
    
    es.bulk(index=index, body=bulk_data, refresh=True)




deleting 'test_migrant' index...
 response: '{u'acknowledged': True}'
creating 'example_index' index...


In [79]:
#test inserts

# following from https://sarahleejane.github.io/learning/python/2015/10/14/creating-an-elastic-search-index-with-python.html
data_dict = [{'parent': '1',
  'prs_achternaam': 'Aafjes',
  'prs_initialen': 'G.H.',
  'prs_tussenvoegsel': '',
  'prs_uuid': '76abc6fc-e8eb-102c-b6de-005056a23d00'},
 {'parent': '2',
  'prs_achternaam': 'Aafjes',
  'prs_initialen': 'K.',
  'prs_tussenvoegsel': '',
  'prs_uuid': '76abd066-e8eb-102c-b6de-005056a23d00'},
 {'parent': '3',
  'prs_achternaam': 'Aay',
  'prs_initialen': 'P.A.',
  'prs_tussenvoegsel': '',
  'prs_uuid': '76abd73c-e8eb-102c-b6de-005056a23d00'},
 {'parent': '4',
  'prs_achternaam': 'Aalbregt',
  'prs_initialen': 'C.',
  'prs_tussenvoegsel': '',
  'prs_uuid': '76abdc82-e8eb-102c-b6de-005056a23d00'},
 {'parent': '5',
  'prs_achternaam': 'Aalderen',
  'prs_initialen': 'K.',
  'prs_tussenvoegsel': 'van',
  'prs_uuid': '76abe13c-e8eb-102c-b6de-005056a23d00'}]


upload_bulk(data_dict)



In [80]:
es.indices.get_mapping(index = index)

{u'test_migrant': {u'mappings': {u'person': {},
   u'person_instance': {u'_parent': {u'type': u'unit'},
    u'_routing': {u'required': True}},
   u'travel': {},
   u'unit': {u'properties': {u'parent': {u'fields': {u'keyword': {u'ignore_above': 256,
        u'type': u'keyword'}},
      u'type': u'text'},
     u'prs_achternaam': {u'type': u'text'},
     u'prs_initialen': {u'type': u'text'},
     u'prs_tussenvoegsel': {u'type': u'text'},
     u'prs_uuid': {u'type': u'text'}}}}}}

In [85]:
result = es.search(body={"query": {"match_all": {}}}, index=index)
print ([hit['_source']  for hit in result['hits']['hits']])

[{u'prs_initialen': u'G.H.', u'prs_tussenvoegsel': u'', u'prs_achternaam': u'Aafjes', u'parent': u'1', u'prs_uuid': u'76abc6fc-e8eb-102c-b6de-005056a23d00'}, {u'prs_initialen': u'P.A.', u'prs_tussenvoegsel': u'', u'prs_achternaam': u'Aay', u'parent': u'3', u'prs_uuid': u'76abd73c-e8eb-102c-b6de-005056a23d00'}, {u'prs_initialen': u'K.', u'prs_tussenvoegsel': u'van', u'prs_achternaam': u'Aalderen', u'parent': u'5', u'prs_uuid': u'76abe13c-e8eb-102c-b6de-005056a23d00'}, {u'prs_initialen': u'K.', u'prs_tussenvoegsel': u'', u'prs_achternaam': u'Aafjes', u'parent': u'2', u'prs_uuid': u'76abd066-e8eb-102c-b6de-005056a23d00'}, {u'prs_initialen': u'C.', u'prs_tussenvoegsel': u'', u'prs_achternaam': u'Aalbregt', u'parent': u'4', u'prs_uuid': u'76abdc82-e8eb-102c-b6de-005056a23d00'}]


In [86]:
def query(index, q):
    """wrapping up arcane elasticsearch results for program reuse
    Kicked out all es metadata for starts
    """
    result = es.search(index=index, body={"query": q})
    res = [hit['_source']  for hit in result['hits']['hits']]
    return res

q = {"match_all": {}}

query(index, q)

[{u'parent': u'1',
  u'prs_achternaam': u'Aafjes',
  u'prs_initialen': u'G.H.',
  u'prs_tussenvoegsel': u'',
  u'prs_uuid': u'76abc6fc-e8eb-102c-b6de-005056a23d00'},
 {u'parent': u'3',
  u'prs_achternaam': u'Aay',
  u'prs_initialen': u'P.A.',
  u'prs_tussenvoegsel': u'',
  u'prs_uuid': u'76abd73c-e8eb-102c-b6de-005056a23d00'},
 {u'parent': u'5',
  u'prs_achternaam': u'Aalderen',
  u'prs_initialen': u'K.',
  u'prs_tussenvoegsel': u'van',
  u'prs_uuid': u'76abe13c-e8eb-102c-b6de-005056a23d00'},
 {u'parent': u'2',
  u'prs_achternaam': u'Aafjes',
  u'prs_initialen': u'K.',
  u'prs_tussenvoegsel': u'',
  u'prs_uuid': u'76abd066-e8eb-102c-b6de-005056a23d00'},
 {u'parent': u'4',
  u'prs_achternaam': u'Aalbregt',
  u'prs_initialen': u'C.',
  u'prs_tussenvoegsel': u'',
  u'prs_uuid': u'76abdc82-e8eb-102c-b6de-005056a23d00'}]